# Pipeline Testing
This file will be steps that will be required to convert uploaded files to MM vector embeddings using voyage-multimodal-3.

## Imports
Importing necessary files and getting important env vars below.

In [ ]:
import os, openai, pinecone-client, pymongo

## Processing Files
Taking uploaded files, converting to screenshot format.
Needs to convert:
- Text into screenshots
- Images into screenshots without too much data
- Audio should be transcribed & chunked, embed text and/or audio chunks
- Video should have key frames sampled, transcribed audio

### Processing Text
I need to find a more effective way to chunk text documents